In [1]:
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import Compose
from torch_geometric.data import Data

import os,sys

In [ ]:
import uproot
import awkward as ak

root_file = "../../data/Dumper_MuGun_FullEta_OneOverPt_1to100_v250514.root"



## Load variable that I'm going to use, from the yml file
import yaml
with open("../configs/dataset_regression.yml", "r") as f:
    config = yaml.safe_load(f)

# Get the list of variables to use
file = uproot.open(root_file)
tree = file[config["tree_name"]]
arr = tree.arrays(library="ak")


[{eventNum: 990004, muonEvent: -2, muonPt: 1.96, muonEta: -0.899, ...}, ...]


In [ ]:
# I want to draw the variables from the array, in particular "stubR": 

def plot_variable(arr, var_name, bins=50):

    # Convertir la variable a un array de NumPy (aplana si es necesario)
    data = ak.to_numpy(arr[var_name])
    if data.ndim > 1:
        data = data.flatten()
    
    print(data)
    plt.figure(figsize=(8,6))
    plt.hist(data, bins=bins, alpha=0.7)
    plt.xlabel(var_name)
    plt.ylabel("Frecuencia")
    plt.title(f"Histograma de {var_name}")
    plt.tight_layout()
    plt.show()


plot_variable(arr, "stubR", bins=100)


ValueError: cannot broadcast records in equal

In [8]:
sys.path.append(os.path.join(os.getcwd(), '..', 'tools', 'training'))
from converter import get_stub_r, get_global_phi, HW_ETA_TO_ETA_FACTOR, remove_empty_or_nan_graphs, getEdgesFromLogicLayer, get_global_phi


if not hasattr(arr, "stubRG"):
    arr['stubRG'] = get_stub_r(arr['stubType'], arr['stubEta'], arr['stubLayer'], arr['stubQuality'])
arr['stubEtaG'] = arr['stubEta'] * HW_ETA_TO_ETA_FACTOR
arr['stubPhiG'] = get_global_phi(arr['stubPhi'], arr['omtfProcessor'])  ## need to check this value!! (not sure it is OK)

if not hasattr(arr, "inputStubRG"):
    arr['inputStubRG'] = get_stub_r(arr['inputStubType'], arr['inputStubEta'], arr['inputStubLayer'], arr['inputStubQuality'])
arr['inputStubEtaG'] = arr['inputStubEta'] * HW_ETA_TO_ETA_FACTOR
arr['inputStubPhiG'] = get_global_phi(arr['inputStubPhi'], arr['omtfProcessor'])  ## need to check this value!! (not sure it is OK)

arr['muonQPt'] = arr['muonCharge'] * arr['muonPt']
arr['muonQOverPt'] = arr['muonCharge'] / arr['muonPt']

ValueError: the truth value of an array whose length is not 1 is ambiguous; use ak.any() or ak.all()